### Sensitive analysis:


In [1]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset
from sktime.classification.kernel_based import RocketClassifier
import time
import numpy as np
import pandas as pd
from sast.sast import znormalize_array,from_2d_array_to_nested
import matplotlib.pyplot as plt
from sktime.datasets import tsc_dataset_names

In [2]:
ds_sens = tsc_dataset_names.univariate_equal_length
ds_sens = ["Chinatown"]
max_ds=len(ds_sens) #exploring dataset in UEA & UCR Time Series Classification Repository
#print(ds_sens)
print(ds_sens)


['Chinatown']


In [15]:
#ds='Chinatown' # Chosing a dataset from # Number of classes to consider
runs = 10
range_max = 100
range_min = 0 
steps= 10

df_result = {}
list_score = []
list_time = []
list_dataset = []
list_hyperparameter = []
list_method = []
for ds in ds_sens:
    X_train, y_train = load_UCR_UEA_dataset(name=ds, extract_path='data', split="train", return_type="numpy2d")
    X_test, y_test = load_UCR_UEA_dataset(name=ds, extract_path='data', split="test", return_type="numpy2d")
    for i in range(runs):
        start = time.time()
        rocket= RocketClassifier(num_kernels=10_000)
        rocket.fit(X_train,y_train)
        end = time.time()
        list_score.append(rocket.score(X_test,y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("num_kernels=10_000")
        list_method.append("Rocket")
        for p in range(range_min,range_max,steps):
            for g in range(range_min,range_max,steps):
                start = time.time()
                random_state = None
                if p ==0:
                    p=1
                if g ==0:
                    g=1
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))
                list_time.append(end-start)
                list_dataset.append(ds)
                list_hyperparameter.append("n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_method.append("Rsast")

    df_result['score']=list_score
    df_result['time']=list_time
    df_result['dataset']=list_dataset
    df_result['hyperparameter']=list_hyperparameter
    df_result['method']=list_method

In [16]:
df_result=pd.DataFrame(df_result)
df_result.to_excel("df_tunning_hyper.xlsx") 
df_result

,score,time,dataset,hyperparameter,method
0,0.982507,7.527988,Chinatown,num_kernels=10_000,Rocket
1,0.793003,0.011937,Chinatown,n_random_points=1 nb_inst_per_class=1,Rsast
2,0.941691,0.043921,Chinatown,n_random_points=1 nb_inst_per_class=10,Rsast
3,0.930029,0.046901,Chinatown,n_random_points=1 nb_inst_per_class=20,Rsast
4,0.924198,0.046873,Chinatown,n_random_points=1 nb_inst_per_class=30,Rsast
...,...,...,...,...,...
96,0.982507,0.747552,Chinatown,n_random_points=90 nb_inst_per_class=50,Rsast
97,0.982507,0.746305,Chinatown,n_random_points=90 nb_inst_per_class=60,Rsast
98,0.982507,0.741306,Chinatown,n_random_points=90 nb_inst_per_class=70,Rsast
99,0.985423,0.746029,Chinatown,n_random_points=90 nb_inst_per_class=80,Rsast
